In [142]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime

driver = webdriver.Chrome('/Users/robjohns/Documents/Metis/Project2/chromedriver')
URL=super_short_test_list[0]
driver.get(URL)
time.sleep(5)
soup = BeautifulSoup(driver.page_source, 'lxml')
driver.quit()
print(len(soup.find_all('div', attrs={'class': 'gws-flights-results__itinerary-duration'})))


30


In [144]:
print(URL)

https://www.google.com/flights?hl=en#flt=SEA.SFO.2019-07-22;b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o


In [ ]:
## how to avoid booting from server ideas . . . .
for page in super_short_test_list:
    ### scrape a website
    ### ...
    print(page)
    
    time.sleep(2)
    #time.sleep(.5+2*random.random())
    
import sys
import requests
from bs4 import BeautifulSoup

url = 'http://www.reddit.com'

user_agent = {'User-agent': 'Mozilla/5.0'}
response  = requests.get(url, headers = user_agent)

from fake_useragent import UserAgent

ua = UserAgent()
user_agent = {'User-agent': ua.random}
print(user_agent)

response  = requests.get(url, headers = user_agent)
print(response.text)

In [151]:
round(2.03333333333333,3)

2.033

In [195]:
##this will take the soupy html file and pull out data values for all flight results from a search##
# looking for [Date,DepartureCity,ArrivalCity,DepartureTime,
#               ArrivalTime,FlightDuration,Price,PlaneType,Airline,FlightNumber]
import re

for li in soup.find_all('div', attrs={'class': 'gws-flights-results__duration flt-subhead1Normal'}):
    chonk=li.get_text()
    try:
        ch=chonk.split('h')
        onk=chonk.split(' ')
        num2=onk[1]
        flight_duration=float(ch[0])+(float(num2[:-1])/60)
        flight_duration = round(flight_duration,2)
        #print ('flight_duration:',flight_duration)
    except:
        print('NaN')

print('')
#for li in soup.find_all('div', attrs={'class': 'gws-flights-results__price'}):
    #print(li)

print('')

for li in soup.find_all('jsl', attrs={'jstcache': "8836"}):
    #print(li.get_text())
    text = li.get_text()
    
    
      #finding price
    try:
        found = re.search('From (.+?)Trip', text).group(1)
        dollar=found.replace(',','')
        found=float(dollar[1:-1])
    except AttributeError:
    # if the sentence structure i didnt expect show up
        found = 'NaN' # apply your error handling
    print('cost:',found)
    
    #finding departure and arrival times
    try:
        found = re.search('Departure time: (.+?)M', text).group(1)
        found2 = re.search('Arrival time: (.+?)M', text).group(1)
        
        #departure=found.replace(',','')
        #found=float(dollar[1:-1])
    except AttributeError:
    # if the sentence structure i didnt expect show up
        found = 'NaN' # apply your error handling
    print('Departure Time:',found)
    print('Arrival Time:',found2)
    
  
    #grab airline
    try:
        found = re.search('by (.+?).Depa', text).group(1)
        
        #departure=found.replace(',','')
        #found=float(dollar[1:-1])
    except AttributeError:
    # if the sentence structure i didnt expect show up
        found = 'NaN' # apply your error handling
    print('Airline:',found)
    
print('')
    #grab PlaneType and flight number
for li in soup.find_all('div', attrs={'class': 'gws-flights-results__other-leg-info'}):
        #print(li.get_text())
        st=li.get_text()
        result = re.match("(?P<PlaneType>[A-Za-z]+.[A-Za-z]*[0-9]+)(?P<FlightNum>[A-Za-z]+\s[0-9]+)", st)
        if result:
            Planetype=result.group('PlaneType')
            Flightnumber=result.group('FlightNum')
            print('Flightnum:',Flightnumber)
            print('Planetype:',Planetype)
            print("")
        else:
            print('no matchy matchy')




cost: 236.0
Departure Time: 6:00 A
Arrival Time: 8:02 A
Airline: United
cost: 236.0
Departure Time: 3:45 P
Arrival Time: 5:55 P
Airline: Delta
cost: 236.0
Departure Time: 9:25 P
Arrival Time: 11:44 P
Airline: Alaska
cost: 236.0
Departure Time: 5:50 A
Arrival Time: 8:15 A
Airline: Alaska
cost: 236.0
Departure Time: 6:00 A
Arrival Time: 8:11 A
Airline: Delta
cost: 236.0
Departure Time: 6:50 A
Arrival Time: 9:13 A
Airline: United
cost: 236.0
Departure Time: 7:40 A
Arrival Time: 10:01 A
Airline: Delta
cost: 236.0
Departure Time: 9:25 A
Arrival Time: 11:48 A
Airline: Delta
cost: 236.0
Departure Time: 9:35 A
Arrival Time: 12:00 P
Airline: Alaska
cost: 236.0
Departure Time: 10:55 A
Arrival Time: 1:20 P
Airline: Alaska
cost: 236.0
Departure Time: 11:15 A
Arrival Time: 1:31 P
Airline: Delta
cost: 236.0
Departure Time: 11:50 A
Arrival Time: 2:15 P
Airline: Alaska
cost: 236.0
Departure Time: 1:45 P
Arrival Time: 3:58 P
Airline: Delta
cost: 236.0
Departure Time: 2:25 P
Arrival Time: 4:45 P
Airli

In [141]:
##Format URLs, can create these lists:      super_short_test_list//test_urls//full_cities_lessdates_urls//
##//full_dates_lesscities_urls//full_url_list//come_home_URLS##

cities=['SFO','AUS','ATL','BOI','BOS','CUN','ORD','DEN','DTW','FAI','JFK','SAN']

##links look like this for one ways,nonstop, with overhead bags allowed, and frontier airlines excluded##
'https://www.google.com/flights?hl=en#flt=SFO.AUS.2019-08-21;b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'

#This makes all the dates from july 22nd to the end of the year, will update from date time later if i have time
dates=['2019-07-22']
for days in range(23,32):
    date = '2019-07-' + str(days)
    dates.append(date)
for days in range(1,32):
    if days>=10:
        date = '2019-08-' + str(days)
        dates.append(date)
    else:
        date = '2019-08-0' + str(days)
        dates.append(date)
for days in range(1,31):
    if days>=10:
        date = '2019-09-' + str(days)
        dates.append(date)
    else:
        date = '2019-09-0' + str(days)
        dates.append(date)
for days in range(1,32):
    if days>=10:
        date = '2019-10-' + str(days)
        dates.append(date)
    else:
        date = '2019-10-0' + str(days)
        dates.append(date) 

for days in range(1,31):
    if days>=10:
        date = '2019-11-' + str(days)
        dates.append(date)
    else:
        date = '2019-11-0' + str(days)
        dates.append(date)           

for days in range(1,32):
    if days>=10:
        date = '2019-12-' + str(days)
        dates.append(date)
    else:
        date = '2019-12-0' + str(days)
        dates.append(date)         

## This makes lists of URLS to use generating data##
super_short_test_list=[]
for city in cities[::6]:
    URLc='https://www.google.com/flights?hl=en#flt=SEA.'+city+'.'
    
    for date in dates[::43]:
        URL=URLc+date+';b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'
        super_short_test_list.append(URL)
print('super short test set has this many values:'+str(len(super_short_test_list)))

test_urls=[]
for city in cities[::5]:
    URLc='https://www.google.com/flights?hl=en#flt=SEA.'+city+'.'
    
    for date in dates[::29]:
        URL=URLc+date+';b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'
        test_urls.append(URL)
print('test set has this many values:'+str(len(test_urls)))        
        
full_cities_lessdates_urls=[]
for city in cities:
    URLc='https://www.google.com/flights?hl=en#flt=SEA.'+city+'.'
    
    for date in dates[::15]:
        URL=URLc+date+';b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'
        full_cities_lessdates_urls.append(URL)
print('full city set has this many values:'+str(len(full_cities_lessdates_urls)))   

full_dates_lesscities_urls=[]
for city in cities[::3]:
    URLc='https://www.google.com/flights?hl=en#flt=SEA.'+city+'.'
    
    for date in dates:
        URL=URLc+date+';b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'
        full_dates_lesscities_urls.append(URL)
print('full date set has this many values:'+str(len(full_dates_lesscities_urls)))  

full_url_list=[]
for city in cities:
    URLc='https://www.google.com/flights?hl=en#flt=SEA.'+city+'.'
    
    for date in dates:
        URL=URLc+date+';b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'
        full_url_list.append(URL)
print('full set has this many values:'+str(len(full_url_list)))  

come_home_URLS=[]
for city in cities:
    URLc='https://www.google.com/flights?hl=en#flt='+city+'.'+'SEA.'
    for date in dates:
        URL=URLc+date+';b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'
        come_home_URLS.append(URL)
print('come home set has this many values:'+str(len(come_home_URLS)))  



test set has this many values:8
test set has this many values:18
full city set has this many values:132
full date set has this many values:652
full set has this many values:1956
come home set has this many values:1956
